# Example data notebook

Content in this page can be created by using Jupyter Notebooks. This means that you can write content very easily, embed images, HTML, etc into your posts!

<div style="text-align:center">
<img src="https://live.staticflickr.com/65535/48992224646_bd32af64ae_k.jpg" width=100%>
</div>

```{seealso}
Check the documentation on the syntax in [here](https://jupyterbook.org/en/stable/content/index.html).
```

## Writing content

The idea for this file is to serve as a data analysis recipe to reproduce your analysis. With both, code, outputs, and data, we can ensure that the findings are actually FAIR.
Let's look at some examples!

## Load data, plot, process, and export
This example will load data from the SmartCitizen API, do some plots, and export it in CSV format

In [ ]:
# Import the Test data structure
from scdata import Test

We can set the framework to output verbose information, and set the rendering framework as jupyterlab

In [ ]:
from scdata._config import config

# Output levels:
# 'QUIET': nothing, 
# 'NORMAL': warn, err
# 'DEBUG': info, warn, err, success
config._out_level='NORMAL' 

# This defaults to script'. If using it in jupyterlab, sets the plot renderers accordingly
config._framework='jupyterlab'

We can put the data in a test. A test is simply a handy way to group devices and process them

In [ ]:
test = Test('EXAMPLE_MINKE')

Before adding devices to the test, we import the Device object:

In [ ]:
from scdata import Device

### Load SC API data

In [ ]:
# Add as many devices as needed. See understanding blueprints below for more info
test.add_device(Device(blueprint = 'sc_21_station_module', descriptor = {'source': 'api', 
                                                                         'id': '14638',
                                                                         'min_date': '2021-10-15',
                                                                         'max_date': '2021-10-30'
                                                             }
                         )
               )

In [ ]:
# Add as many devices as needed. See understanding blueprints below for more info
test.add_device(Device(blueprint = 'sck_21', descriptor = {'source': 'api', 
                                                                         'id': '14735', 
                                                                         'max_date': '2022-02-15',
                                                             }
                         )
               )

The blueprint urls can be found at `~/.config/scdata/config.yaml` or here in:


In [ ]:
config.blueprints

In [ ]:
config.blueprints.keys()

This is how you can add a csv device (from sdcard data or other). The raw data file is to be put in `~/.cache/scdata/raw`:

### Load SD Card Data

In [ ]:
test.add_device(Device(blueprint = 'sck_21' , descriptor = {'source': 'csv',
                                                                  'id': 'csv_device',
                                                                  'raw_data_file': 'example.csv',
                                                                  'frequency': '1Min',
                                                                  'timezone': 'Europe/Madrid'
                                                         }
                     )
           )

This instruction will create the test folder structure, the description and anything needed to keep track of the data:

In [ ]:
# Create the test
test.create()

In [ ]:
# Load it
test.load()

In [ ]:
# Check the devices
test.devices

In [ ]:
# Check the data
test.devices['14638'].readings

## Plotting data

There are many options for plotting data. Here you can find some basic timeseries examples:

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": "NOISE_A",
                  "subplot": 1},
        }

figure = test.ts_plot(traces = traces, 
                      options = {'frequency': '10Min', 'min_date': '2022-01'})

In [ ]:
traces = {
            "1": {"devices": 'all',
                  "channel": "TEMP",
                  "subplot": 1},
            "2": {"devices": 'all',
                  "channel": "NOISE_A",
                  "subplot": 2},
        }

height = 300

f = test.ts_uplot(traces = traces, 
              formatting = {'width': height*2.5, 
                            'height': height, 
                            'padding-bottom': height,
                            'title': 'Temperature', 
                            'ylabel': {1: 'Temperature (degC)',
                                       2: 'Noise (dBA)'}
                           }, 
              options = {'frequency': '10Min', 'html': False});


In [ ]:
f

```{seealso}
For more examples, visit: https://github.com/fablabbcn/smartcitizen-data/blob/master/examples/notebooks/03_data_visualisation.ipynb
```

## Understanding blueprints

Blueprints are ways of defining devices that include all the metadata necessary for that device, regarding units, processes needed to clean the data, file locations, even the documentation of the sensors. Blueprints data come from urls and are loaded while initialising the `scdata`. In the `~/.config/scdata/config.yaml` file more urls can be defined and can be accessed by:

In [ ]:
config.blueprints

In [ ]:
config.blueprints['sck_21']

Each of the properties in a device is defined in it's blueprint. For instance, the sensors available. 

In [ ]:
config.blueprints['sck_21']['sensors']

In the case of devices coming from the SC platform, these sensors are defined based on [these definitions](https://api.smartcitizen.me/v0/kits), and are meant for traceability of the data, explaining what each device contains. For other devices, this has to be filled out manually.

The bluepring also contains the metrics (or processed channels) associated with the sensors. These metrics will be calculated once test.process() or device.process() are called

In [ ]:
config.blueprints['sck_21']['metrics']

Finally, to add your own blueprint, you can add the url directly in the `config.yaml` file or by:

In [ ]:
config.blueprints_urls.append('another_url')
config.get_meta_data()

## Data structure
Here we show how the data is structured. A test contains devices, metadata and models (if created, see sensor_calibration_workflows.ipynb)

First the tests:

In [ ]:
list(test.devices.keys())

Then the data inside the devices is simply a pandas.DataFrame()

In [ ]:
test.devices['14638'].readings.head(5)

The test description information can be accessed and modified in the `test.descriptor` `dict()`. You can also access where the data is with `test.path`

In [ ]:
test.descriptor

## Exporting data

Finally, we can export the whole test as above (by default in the `test` directory) or just a device in a defined path

In [ ]:
# test.devices['14638'].export(path ='~/Desktop')

Export the data and test descriptor file

In [ ]:
test.to_html(title='Example');

Or save the whole test in the default test processed folder with `test.to_csv()`